## Mission statement

* Cassandra is a **wide-column database**.
* Wide-column databases are two-dimensional key-value stores, where each key corresponds with a set of columns. The in-memory equivalent would be a `dict` inside a `dict` in Python.
* They differ from a true relational database in that the sub-keys can be anything. You are not forced to use the specific columns in the table schema, and instead can use any columns and values that strike your fancy.
* Meanwhile, while true column stores provide locality on all of the columns, wide-column data stores provide locality on the individual records.
* They're effectively schemaless transactional databases.
* Cassandra itself is designed for partition tolerance and availability, but not consistency.
* It's designed to be sharded heavily and to deal with truly big "big data" distributions.

## Data model

* Cassandra arranges its clusters using a shared-nothing **ring architecture**. Each node is in communication with a node immediately to its left and to its right, but doesn't explicitly depend on the availability of its peers.
* There are no masters or slaves, only peers.
* Clients access specific entry nodes, but data is asynchronously replicated across the nodes. If the desired node goes down, the client can cycle to other nodes, which should still be up (achieving availability).
* This architecture is lifted directly from the Dynamo system, which came slightly beforehand.


* The Cassandra data model is a standard SSTable LSM-tree implementation, taken from BigTable. So: write to commit log, write to memtable, acknowledge to client, periodically flush the memtable into log files, periodicially merge logfiles into a new unified log.
* A hash is used to determine which node in the ring will accept the write for a chunk of data.
* After the insertion operation is finished, replication is done by sending the data to the left-right nodes the nodes is in communication.
* You can configure more replicating by specifying the **replication factor**.


* On a read, the client connects to any node they want (or any node that's available, really). The node services the request by internally routing to nodes in the cluster which have the data.
* Due to network partitions and the asynchronous nature of data sharing, different nodes may have data that is in different states of recency.
* Thus you can tune how far to look for a response, depending on the level of consistency that you desire.
* A minimum level of consistency, `ONE`, will result in the *first* node that has the data reporting that data to the client.
* On the flip side, the maximum level of consistency, `ALL`, will result in *every* node that has that data shared with it reporting that data. The data that has the most recent timestamp will be the data that is reported.
* Most commonly you want `QUORUM`. In this case, 51% of nodes report, and the most recently timestamped data point amongst these nodes is returned.
* Thus in Cassandra there is a trade-off between *consistency* and *speed*.
* If only one node needs to report data on read, then Cassandra is highly available, but not highly consistent (there can be laspes in data sameness). If full consensus is used, then Cassandra is highly consistent, but not highly available (what happens if a node goes down?).
* Thus Cassandra offers tunable consistency.


* This model has linear scaling performance. The Cassandra database architecture scales better than basically any other database architecture out there, making it a preferred solution for truly "big data" problems.


* Queries are via CQL.